# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.13.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

- apply tranformations to the images to avoid overfitting
- if not applying transformations, the model will learn the images as it is and will not generalize well
- these transformations are like rotating, flipping, zooming, etc and are called data augmentation
- overfitting is model memorizing the training data and not generalizing well to new data which is test data

In [3]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255, # Feature scaling (normalization) to make the values between 0 and 1 
        shear_range=0.2, # Shear Intensity (Shear angle in counter-clockwise direction in degrees) 
        zoom_range=0.2,     # Zoom Intensity (Range for random zoom) 
        horizontal_flip=True) # Horizontal Flip (Boolean, Randomly flip inputs horizontally) 

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
training_set= train_datagen.flow_from_directory( # Generates batches of tensor image data with real-time data augmentation 
        '/Users/nisankarsan/Downloads/Codes/Deep Learning A-Z/Part 2 - Convolutional Neural Networks (CNN)/dataset/training_set', 
        # directory of the training data 
        target_size=(64, 64),  # dimensions to which all images found will be resized 
        batch_size=32, # size of the batches of data (default: 32), batch size is the number of training examples utilized in one iteration
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels 

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
# this is the augmentation configuration we will use for testing:
# only rescaling (normalization) because we don't want to augment the test data 
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a similar generator, for validation data for the model 
test_set = test_datagen.flow_from_directory('/Users/nisankarsan/Downloads/Codes/Deep Learning A-Z/Part 2 - Convolutional Neural Networks (CNN)/dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential() # Initialising the CNN which is a sequence of layers 

2024-11-12 17:05:51.056952: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-11-12 17:05:51.057006: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-11-12 17:05:51.057016: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-11-12 17:05:51.057058: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-12 17:05:51.057078: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3])) # Convolutional Layer


# filters: number of filters in the convolution , kernel_size: dimensions of the convolution window, activation: activation function
#Conv2D is the layer to convolve the image into multiple images 
# input_shape: shape of the input image (64x64 pixels and 3 channels for colored images) if the image is black and white, the input_shape will be [64, 64, 1]
# Rectified Linear Unit (ReLU) activation function is used to add non-linearity to the model
# ReLU is used to convert all the negative pixel values to zero
# The first layer of the CNN is the Convolutional Layer which is used to detect features in the input image
# The input image is passed through a filter (kernel) to produce a feature map
# The feature map highlights the features in the input image
# The Convolutional Layer is the building block of the CNN
# The Convolutional Layer is followed by the Pooling Layer



### Step 2 - Pooling

- The Pooling Layer is used to reduce the spatial dimensions of the Convolutional Layer
- The Pooling Layer is used to reduce the computational complexity of the CNN

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) # Pooling Layer
# pool_size: dimensions of the pooling window, strides: factor by which the window shifts
# MaxPooling is used to reduce the spatial dimensions of the Convolutional Layer

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu')) # Convolutional Layer
#remove input_shape as it is not required for the subsequent layers 
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))# Pooling Layer



### Step 3 - Flattening

- flattening is the process of converting all the resultant 2D arrays into a single long continuous linear vector


In [9]:
cnn.add(tf.keras.layers.Flatten()) # Flattening Layer
# Flatten is used to convert the 2D feature maps into a single 1D vector


### Step 4 - Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu')) # Fully Connected Layer
# units: number of hidden neurons in the layer, activation: activation function is ReLU used to add non-linearity to the model 
# Dense is used to connect all the neurons of the previous layer to the next layer
# The Fully Connected Layer is used to connect the neurons of the previous layer to the next layer
# The Fully Connected Layer is the last layer of the CNN


### Step 5 - Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) 

 # Output Layer
# units: number of output neurons in the layer which is 1 cat or dog,
# activation: activation function is Sigmoid used to predict the class labels of the input image 
# The Output Layer is used to predict the class labels of the input image 
# The Output Layer is the last layer of the CNN


## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Compiling the CNN
# optimizer: Adam is used to update the weights of the network
# loss: binary_crossentropy is used for binary classification dog or cat
# metrics: accuracy is used to evaluate the performance of the model
# The compile method is used to configure the CNN for training 

### Training the CNN on the Training set and evaluating it on the Test set

what is the \
Epoch 1/25 \
250/250 \
we have 8000 images and also batch size =32 so 8000/32= 250 which means have 250 steps in each epoch to reach to total amount of 8000 images


In [13]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)
# x: training data, validation_data: validation data, epochs: number of iterations over the entire training data


Epoch 1/25


2024-11-12 17:05:51.547325: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - ETA: 0s - loss: 0.6675 - accuracy: 0.5896

2024-11-12 17:06:05.140218: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 16s 64ms/step - loss: 0.6675 - accuracy: 0.5896 - val_loss: 0.8057 - val_accuracy: 0.5480
Epoch 2/25
250/250 [==============================] - 16s 63ms/step - loss: 0.6123 - accuracy: 0.6665 - val_loss: 0.5712 - val_accuracy: 0.7075
Epoch 3/25
250/250 [==============================] - 16s 63ms/step - loss: 0.5775 - accuracy: 0.6981 - val_loss: 0.6009 - val_accuracy: 0.6755
Epoch 4/25
250/250 [==============================] - 16s 63ms/step - loss: 0.5686 - accuracy: 0.7088 - val_loss: 0.5869 - val_accuracy: 0.6970
Epoch 5/25
250/250 [==============================] - 15s 62ms/step - loss: 0.5796 - accuracy: 0.6999 - val_loss: 0.6201 - val_accuracy: 0.6985
Epoch 6/25
250/250 [==============================] - 15s 58ms/step - loss: 0.6034 - accuracy: 0.6966 - val_loss: 0.6667 - val_accuracy: 0.6830
Epoch 7/25
250/250 [==============================] - 15s 60ms/step - loss: 0.6061 - accuracy: 0.7010 - val_loss: 0.6065 - val_accuracy: 0.7155
Epo

## Part 4 - Making a single prediction

In [47]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('/Users/nisankarsan/Downloads/Codes/Deep Learning A-Z/Part 2 - Convolutional Neural Networks (CNN)/dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 [==============================] - 0s 11ms/step


In [48]:
print(prediction)

dog
